In [26]:
import pandas as pd
pd.set_option('display.max_columns', 500)
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as MSE
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn import preprocessing
import numpy as np

In [2]:
loan = pd.read_csv(
    'Loan_Data.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loan.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32
0,11.89%,54734.0,80364.0,"$25,000","$25,000","$19,080",36 months,B,B4,NaN,< 1 year,RENT,85000.0,VERIFIED - income,Aug-09,Due to a lack of personal finance education an...,debt_consolidation,Debt consolidation for on-time payer,941xx,CA,19.48,0.0,Feb-94,0.0,NaN,NaN,10.0,0.0,28854.0,52.10%,42.0,f
1,10.71%,55742.0,114426.0,"$7,000","$7,000",$673,36 months,B,B5,CNN,< 1 year,RENT,65000.0,not verified,May-08,Just want to pay off the last bit of credit ca...,credit_card,Credit Card payoff,112xx,NY,14.29,0.0,Oct-00,0.0,NaN,NaN,7.0,0.0,33623.0,76.70%,7.0,f
2,16.99%,57167.0,137225.0,"$25,000","$25,000","$24,725",36 months,D,D3,Web Programmer,1 year,RENT,70000.0,VERIFIED - income,Aug-14,Trying to pay a friend back for apartment brok...,debt_consolidation,mlue,100xx,NY,10.50,0.0,Jun-00,0.0,41.0,NaN,10.0,0.0,19878.0,66.30%,17.0,f
3,13.11%,57245.0,138150.0,"$1,200","$1,200","$1,200",36 months,C,C2,city of beaumont texas,10+ years,OWN,54000.0,not verified,Mar-10,"If funded, I would use this loan consolidate t...",debt_consolidation,zxcvb,777xx,TX,5.47,0.0,Jan-85,0.0,64.0,NaN,5.0,0.0,2584.0,40.40%,31.0,f
4,13.57%,57416.0,139635.0,"$10,800","$10,800","$10,692",36 months,C,C3,State Farm Insurance,6 years,RENT,32000.0,not verified,Nov-09,I currently have a personal loan with Citifina...,debt_consolidation,Nicolechr1978,067xx,CT,11.63,0.0,Dec-96,1.0,58.0,NaN,14.0,0.0,3511.0,25.60%,40.0,f


In [4]:
loan.shape

(400000, 32)

# Cleaning

In [3]:
loan['X1'] = pd.to_numeric(loan['X1'].str.replace('%', ''))/100

In [8]:
loan.head(1)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32
0,0.1189,54734.0,80364.0,"$25,000","$25,000","$19,080",36 months,B,B4,NaN,< 1 year,RENT,85000.0,VERIFIED - income,Aug-09,Due to a lack of personal finance education an...,debt_consolidation,Debt consolidation for on-time payer,941xx,CA,19.48,0.0,Feb-94,0.0,NaN,NaN,10.0,0.0,28854.0,52.10%,42.0,f


In [4]:
# get the number of missing data points per column
missing_values_count = loan.isnull().sum().sort_values(ascending=False)

percent_1 = loan.isnull().sum()/loan.isnull().count()*100
percent_2 = round(percent_1,1).sort_values(ascending=False)

missing_data = pd.concat([missing_values_count,percent_2],axis='columns',keys=['Total','%'],sort=False)

# look at the # of missing points in the first ten columns
missing_data

,Total,%
X26,348845,87.2
X16,276440,69.1
X25,218802,54.7
X12,61361,15.3
X8,61270,15.3
X9,61270,15.3
X13,61028,15.3
X1,61010,15.3
X10,23986,6.0
X11,17538,4.4


In [13]:
loan['X11'].value_counts()

10+ years    91815
2 years      25541
3 years      22599
< 1 year     22009
5 years      19506
1 year       18116
4 years      17207
6 years      16565
7 years      16435
8 years      13765
9 years      10942
Name: X11, dtype: int64

In [5]:
loan['X8'].value_counts()

B    101668
C     90071
D     55621
A     53707
E     25518
F      9784
G      2361
Name: X8, dtype: int64

In [6]:
loan['X9'].value_counts()

B3    24009
B4    22611
B2    19853
C1    19285
C2    19182
B5    18301
C3    18244
C4    17225
B1    16894
C5    16135
A5    15485
D1    13840
A4    13837
D2    12479
D3    10818
D4    10131
A3     9031
D5     8353
A2     7678
A1     7676
E1     6408
E2     6181
E3     4978
E4     4291
E5     3660
F1     2878
F2     2202
F3     2022
F4     1537
F5     1145
G1      790
G2      598
G3      428
G4      300
G5      245
Name: X9, dtype: int64

In [4]:
# X4 = Loan amount requested
# X30 = Revolving line utilization rate
# X8 = Loan grade
# X9 = Loan subgrade
# X1 = Interest Rate on the loan
# X13 = Annual income of borrower
# X11 = Number of years employed (0 to 10; 10 = 10 or more)
loan.dropna(axis='rows', subset=['X4','X30','X8','X9','X1','X13','X11'], inplace=True)

In [6]:
# get the number of missing data points per column
missing_values_count = loan.isnull().sum().sort_values(ascending=False)

percent_1 = loan.isnull().sum()/loan.isnull().count()*100
percent_2 = round(percent_1,1).sort_values(ascending=False)

missing_data = pd.concat([missing_values_count,percent_2],axis='columns',keys=['Total','%'],sort=False)

# look at the # of missing points in the first ten columns
missing_data.head(7)

,Total,%
X26,203638,87.6
X16,159907,68.8
X25,126912,54.6
X12,35759,15.4
X10,4004,1.7
X18,11,0.0
X8,0,0.0


In [19]:
loan['X17'].value_counts()

debt_consolidation    136381
credit_card            51620
home_improvement       13318
other                  11611
major_purchase          4984
small_business          3770
car                     2833
medical                 2238
moving                  1462
wedding                 1363
vacation                1239
house                   1199
educational              209
renewable_energy         186
Name: X17, dtype: int64

In [20]:
loan['X23'].value_counts()

Oct-00    1893
Aug-00    1816
Oct-99    1779
Oct-01    1741
Nov-99    1724
Aug-01    1706
Nov-00    1689
Sep-99    1633
Sep-00    1611
Aug-99    1583
Sep-01    1572
Dec-98    1571
Nov-98    1561
Oct-02    1548
Dec-99    1498
Dec-00    1472
May-00    1458
Jul-00    1457
Jan-01    1456
Aug-02    1454
Oct-98    1453
May-01    1450
Jul-01    1439
Nov-01    1437
Sep-02    1424
Jan-00    1403
Dec-01    1399
Mar-00    1396
Jun-01    1388
Sep-98    1373
          ... 
Dec-60       1
Mar-60       1
Jan-46       1
Nov-50       1
Aug-60       1
Jan-57       1
Jun-49       1
Oct-57       1
Dec-58       1
May-50       1
Dec-61       1
Nov-63       1
Jan-60       1
Nov-56       1
Aug-58       1
Nov-62       1
Oct-59       1
Feb-65       1
Jan-44       1
Dec-50       1
Aug-62       1
Sep-11       1
Oct-50       1
Nov-53       1
May-64       1
Jul-63       1
Sep-63       1
Aug-63       1
May-62       1
May-58       1
Name: X23, Length: 619, dtype: int64

In [22]:
loan['X32'].value_counts()

f    159751
w     72662
Name: X32, dtype: int64

In [5]:
# X2 = unique id for the loan
# X3 = unique id assigned for the borrower
# X16 = reason for loan provided by borrower
# X10 = employer or job title (self-filled)
# X18 = loan title, as provided by borrower
# X25 = number of months since the borrower's last delinquency
# X26 = number of months since the last public record
# X12 = Home ownership status: RENT, OWN, MORTGAGE, OTHER.
# X15 = Date loan was issued
# X17 = Loan category, as provided by borrower
# X23 = Date the borrower's earliest reported credit line was opened
# X32 = The initial listing status of the loan
# X11 = Number of years employed (0 to 10; 10 = 10 or more)
# X20 = State of borrower
loan.drop(columns=['X2','X3','X16','X10','X18','X25','X26','X12','X15','X17','X23',
                  'X11','X20'], inplace=True)

#### Note: Check the density using .value_counts for categorical variables and perform one-hot encoding for the top variables

In [8]:
# get the number of missing data points per column
missing_values_count = loan.isnull().sum().sort_values(ascending=False)

percent_1 = loan.isnull().sum()/loan.isnull().count()*100
percent_2 = round(percent_1,1).sort_values(ascending=False)

missing_data = pd.concat([missing_values_count,percent_2],axis='columns',keys=['Total','%'],sort=False)

# look at the # of missing points in the first ten columns
missing_data.head()

,Total,%
X32,0,0.0
X14,0,0.0
X4,0,0.0
X5,0,0.0
X6,0,0.0


In [6]:
loan['X4'] = loan['X4'].str.replace('$', '')
loan['X4'] = pd.to_numeric(loan['X4'].str.replace(',', ''))
loan['X5'] = loan['X5'].str.replace('$', '')
loan['X5'] = pd.to_numeric(loan['X5'].str.replace(',', ''))
loan['X6'] = loan['X6'].str.replace('$', '')
loan['X6'] = pd.to_numeric(loan['X6'].str.replace(',', ''))
loan['X7'] = pd.to_numeric(loan['X7'].str.replace(' months', ''))
loan['X19'] = pd.to_numeric(loan['X19'].str.replace('x', ''))
loan['X21'] = loan['X21']/100 
loan['X13'] = loan['X13'].astype(int) 
loan['X22'] = loan['X22'].astype(int) 
loan['X24'] = loan['X24'].astype(int)
loan['X27'] = loan['X27'].astype(int)
loan['X28'] = loan['X28'].astype(int)
loan['X29'] = loan['X29'].astype(int)
loan['X30'] = loan['X30'].str.replace('%', '')
loan['X30'] = pd.to_numeric(loan['X30'].str.replace(',', ''))/100
loan['X31'] = loan['X31'].astype(int) 

loan['X8'] = preprocessing.LabelEncoder().fit_transform(loan['X8'].values)
loan['X9'] = preprocessing.LabelEncoder().fit_transform(loan['X9'].values)
loan['X14'] = preprocessing.LabelEncoder().fit_transform(loan['X14'].values)
loan['X32'] = preprocessing.LabelEncoder().fit_transform(loan['X32'].values)

In [7]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232413 entries, 0 to 399999
Data columns (total 19 columns):
X1     232413 non-null float64
X4     232413 non-null int64
X5     232413 non-null int64
X6     232413 non-null int64
X7     232413 non-null int64
X8     232413 non-null int64
X9     232413 non-null int64
X13    232413 non-null int64
X14    232413 non-null int64
X19    232413 non-null int64
X21    232413 non-null float64
X22    232413 non-null int64
X24    232413 non-null int64
X27    232413 non-null int64
X28    232413 non-null int64
X29    232413 non-null int64
X30    232413 non-null float64
X31    232413 non-null int64
X32    232413 non-null int64
dtypes: float64(3), int64(16)
memory usage: 35.5 MB


In [14]:
loan.head()

,X1,X4,X5,X6,X7,X8,X9,X13,X14,X19,X21,X22,X24,X27,X28,X29,X30,X31,X32
0,0.1189,25000,25000,19080,36,1,8,85000,0,941,0.1948,0,0,10,0,28854,0.521,42,0
1,0.1071,7000,7000,673,36,1,9,65000,2,112,0.1429,0,0,7,0,33623,0.767,7,0
2,0.1699,25000,25000,24725,36,3,17,70000,0,100,0.1050,0,0,10,0,19878,0.663,17,0
3,0.1311,1200,1200,1200,36,2,11,54000,2,777,0.0547,0,0,5,0,2584,0.404,31,0
4,0.1357,10800,10800,10692,36,2,12,32000,2,67,0.1163,0,1,14,0,3511,0.256,40,0


# Machine Learning

In [8]:
X = loan.drop(['X1'], axis='columns')

In [9]:
y = loan.X1

In [16]:
X.head()

,X4,X5,X6,X7,X8,X9,X13,X14,X19,X21,X22,X24,X27,X28,X29,X30,X31,X32
0,25000,25000,19080,36,1,8,85000,0,941,0.1948,0,0,10,0,28854,0.521,42,0
1,7000,7000,673,36,1,9,65000,2,112,0.1429,0,0,7,0,33623,0.767,7,0
2,25000,25000,24725,36,3,17,70000,0,100,0.1050,0,0,10,0,19878,0.663,17,0
3,1200,1200,1200,36,2,11,54000,2,777,0.0547,0,0,5,0,2584,0.404,31,0
4,10800,10800,10692,36,2,12,32000,2,67,0.1163,0,1,14,0,3511,0.256,40,0


In [17]:
y.head()

0    0.1189
1    0.1071
2    0.1699
3    0.1311
4    0.1357
Name: X1, dtype: float64

## Training

In [18]:
#split into 75% training, 25% test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size = 0.25)

In [19]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(174309, 18)
(58104, 18)
(174309,)
(58104,)


### XGBOOST

In [20]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [21]:
param_dict = {
    'max_depth':6, 
    'min_child_weight': 1,
    'eta':.1,
    'subsample': 1,
    'colsample_bytree': 1,
    'objective':'reg:linear',
    'eval_metric':'rmse'
}

In [22]:
num_boost_round=999
model = xgb.train(param_dict, dtrain, num_boost_round=num_boost_round, evals=[(dtest, "Test")],
                  early_stopping_rounds=10)

[17:22:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[0]	Test-rmse:0.326952
Will train until Test-rmse hasn't improved in 10 rounds.
[17:22:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[1]	Test-rmse:0.294284
[17:22:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[2]	Test-rmse:0.264885
[17:22:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[3]	Test-rmse:0.238429
[17:22:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=6
[4]	Test-rmse:0.214621
[17:22:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=6
[5]	Test-rmse:0.193196
[17:22:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=6
[6]	Test-rmse:0.17391

[17:22:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[60]	Test-rmse:0.007509
[17:22:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[61]	Test-rmse:0.007494
[17:22:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[62]	Test-rmse:0.007484
[17:22:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes, max_depth=6
[63]	Test-rmse:0.007466
[17:22:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[64]	Test-rmse:0.00746
[17:22:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[65]	Test-rmse:0.007455
[17:22:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[66]	Test-rmse:0.007451
[17:22:50] src/tree/updater_prune.cc:74: tr

[17:22:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[120]	Test-rmse:0.007211
[17:22:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[121]	Test-rmse:0.007205
[17:22:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=6
[122]	Test-rmse:0.007204
[17:22:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[123]	Test-rmse:0.007204
[17:22:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[124]	Test-rmse:0.007199
[17:22:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[125]	Test-rmse:0.007196
[17:22:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=6
[126]	Test-rmse:0.007193
[17:22:56] src/tree/updater_prune.c

[179]	Test-rmse:0.007134
[17:23:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[180]	Test-rmse:0.00713
[17:23:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[181]	Test-rmse:0.007129
[17:23:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[182]	Test-rmse:0.007129
[17:23:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[183]	Test-rmse:0.007129
[17:23:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 0 pruned nodes, max_depth=6
[184]	Test-rmse:0.007129
[17:23:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[185]	Test-rmse:0.007126
[17:23:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[186]	Test-rmse:0.007123
[17:23:01] 

[17:23:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[240]	Test-rmse:0.007086
[17:23:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[241]	Test-rmse:0.007086
[17:23:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=6
[242]	Test-rmse:0.007086
[17:23:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[243]	Test-rmse:0.007085
[17:23:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[244]	Test-rmse:0.007085
[17:23:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[245]	Test-rmse:0.007085
[17:23:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[246]	Test-rmse:0.007084
[17:23:08] src/tree/updater_prune.cc

In [32]:
xgboost_accuracy = model.best_score
xgboost_accuracy

0.007063

In [34]:
dpred=xgb.DMatrix(X)

xgboost_RMSE = MSE(model.predict(dpred), y)**(1/2)
xgboost_RMSE

0.006733399164007152

### Gradient Boosting

In [64]:
gradient_boosting_model = GradientBoostingRegressor(max_depth=4, n_estimators=200,random_state=2)
gradient_boosting_model.fit(X_train, y_train)
gradient_boosting_y_pred = gradient_boosting_model.predict(X_test)

gradient_boosting_model.score(X_test,y_test)

0.9722166431424528

In [65]:
gradient_boosting_mse_test = MSE(y_test, gradient_boosting_y_pred)

# Compute RMSE
gradient_boosting_rmse_test = gradient_boosting_mse_test**(1/2)
gradient_boosting_rmse_test

0.007322890670520562

## Holdout Data

In [58]:
loan_holdout = pd.read_csv(
    'Holdout_for_Testing.csv')

In [55]:
loan_holdout.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32
0,NaN,44409194,47416907,"$6,000","$6,000","$6,000",36 months,C,C5,Electrician,10+ years,MORTGAGE,68000.0,VERIFIED - income,15-Mar,NaN,debt_consolidation,Debt consolidation,430xx,OH,28.31,0,2-Nov,1,26.0,NaN,18,0,19861,64.50%,33,f
1,NaN,44017917,47034722,"$24,000","$24,000","$24,000",36 months,A,A1,Executive Assistant,8 years,RENT,110480.0,VERIFIED - income,15-Mar,NaN,debt_consolidation,Debt consolidation,945xx,CA,16.03,0,Dec-68,1,NaN,NaN,12,0,17001,26.20%,36,w
2,NaN,44259158,47306871,"$35,000","$35,000","$35,000",36 months,C,C2,District Sales Leader,10+ years,MORTGAGE,86000.0,VERIFIED - income,15-Mar,NaN,debt_consolidation,Debt consolidation,195xx,PA,32.49,0,Oct-98,0,NaN,NaN,16,0,25797,49.90%,33,w
3,NaN,44429213,47476932,"$10,000","$10,000","$10,000",60 months,D,D1,pharmacy associate,10+ years,RENT,30000.0,VERIFIED - income source,15-Mar,NaN,debt_consolidation,Debt consolidation,921xx,CA,32.96,0,Feb-99,1,NaN,114.0,13,1,9586,43.80%,21,w
4,NaN,44299188,47346901,"$24,000","$24,000","$24,000",60 months,B,B1,Medical case manager,10+ years,MORTGAGE,82500.0,VERIFIED - income,15-Mar,NaN,debt_consolidation,Debt consolidation,196xx,PA,31.03,0,2-Dec,0,48.0,NaN,27,0,31842,41.30%,43,w


### Cleaning

In [25]:
# get the number of missing data points per column
missing_values_count = loan_holdout.isnull().sum().sort_values(ascending=False)

percent_1 = loan_holdout.isnull().sum()/loan_holdout.isnull().count()*100
percent_2 = round(percent_1,1).sort_values(ascending=False)

missing_data = pd.concat([missing_values_count,percent_2],axis='columns',keys=['Total','%'],sort=False)

# look at the # of missing points in the first ten columns
missing_data

,Total,%
X1,80000,100.0
X16,79985,100.0
X26,66161,82.7
X25,38704,48.4
X10,4394,5.5
X11,4382,5.5
X30,30,0.0
X14,0,0.0
X2,0,0.0
X3,0,0.0


In [59]:
loan_holdout.dropna(axis='rows', subset=['X4','X30','X8','X9','X13','X11'], inplace=True)

loan_holdout.drop(columns=['X2','X3','X16','X10','X18','X25','X26','X12','X15','X17','X23',
                  'X11','X20','X1'], inplace=True)

loan_holdout['X4'] = loan_holdout['X4'].str.replace('$', '')
loan_holdout['X4'] = pd.to_numeric(loan_holdout['X4'].str.replace(',', ''))
loan_holdout['X5'] = loan_holdout['X5'].str.replace('$', '')
loan_holdout['X5'] = pd.to_numeric(loan_holdout['X5'].str.replace(',', ''))
loan_holdout['X6'] = loan_holdout['X6'].str.replace('$', '')
loan_holdout['X6'] = pd.to_numeric(loan_holdout['X6'].str.replace(',', ''))
loan_holdout['X7'] = pd.to_numeric(loan_holdout['X7'].str.replace(' months', ''))
loan_holdout['X19'] = pd.to_numeric(loan_holdout['X19'].str.replace('x', ''))
loan_holdout['X21'] = loan_holdout['X21']/100 
loan_holdout['X13'] = loan_holdout['X13'].astype(int) 
loan_holdout['X22'] = loan_holdout['X22'].astype(int) 
loan_holdout['X24'] = loan_holdout['X24'].astype(int)
loan_holdout['X27'] = loan_holdout['X27'].astype(int)
loan_holdout['X28'] = loan_holdout['X28'].astype(int)
loan_holdout['X29'] = loan_holdout['X29'].astype(int)
loan_holdout['X30'] = loan_holdout['X30'].str.replace('%', '')
loan_holdout['X30'] = pd.to_numeric(loan_holdout['X30'].str.replace(',', ''))/100
loan_holdout['X31'] = loan_holdout['X31'].astype(int) 

loan_holdout['X8'] = preprocessing.LabelEncoder().fit_transform(loan_holdout['X8'].values)
loan_holdout['X9'] = preprocessing.LabelEncoder().fit_transform(loan_holdout['X9'].values)
loan_holdout['X14'] = preprocessing.LabelEncoder().fit_transform(loan_holdout['X14'].values)
loan_holdout['X32'] = preprocessing.LabelEncoder().fit_transform(loan_holdout['X32'].values)

### Machine Learning

In [61]:
y_pred = model.predict(xgb.DMatrix(loan_holdout))

In [62]:
predict = pd.DataFrame(y_pred, columns=['Predictions'])

predict.to_csv("Results from Alvin Haryanto.csv", index=False)

# Observation

I used two boosting models to train the data for loan prediction, they are XGBOOST and gradient boosting. XGBOOST produced lower RMSE compared to gradient boosting and therefore is a better model.